In [1]:
import json
import numpy as np

from functools import reduce
from uuid import uuid4

# typing imports
from typing import Literal, Annotated
from nptyping import NDArray, Shape, Int
from beartype import beartype
from beartype.vale import Is


T_GRID = NDArray[Shape["*, *"], Int]


@beartype
def reiterate_grid(grid: T_GRID) -> T_GRID:
    flat_grid = grid.flatten()
    unique_ids, unique_inds = np.unique(flat_grid, return_index=True)

    # sort the unique ids by their unique indices
    dtype = [("id", int), ("ind", int)]  # create a dtype for sorting purposes
    unique_grid = np.array(list(zip(unique_ids, unique_inds)), dtype=dtype)
    sorted_grid = np.sort(unique_grid, order="ind")

    new_ids = np.arange(sorted_grid.size)  # have new ids be monotonically increasing
    id_map = {old: new for (old, _), new in zip(sorted_grid, new_ids)}
    new_grid = np.array([id_map[old] for old in flat_grid])

    return new_grid.reshape(grid.shape)

T_POS_INT = Annotated[int, Is[lambda x: x > 0]]
T_STRATEGY = Literal["outer", "inner", "front", "back"]
T_PERCS = NDArray[Shape["*"], Int]


@beartype
def get_int_percs(n_parts: T_POS_INT, strategy: T_STRATEGY = "outer") -> T_PERCS:
    TEN_K = 10_000
    perc = TEN_K // n_parts
    n_perc_pp = TEN_K % perc
    percs_base = np.full(n_parts, perc)
    percs_incr = np.ones(n_perc_pp)
    percs_same = np.zeros(n_parts - n_perc_pp)
    match strategy:
        case "outer":
            incr_parts = np.array_split(percs_incr, 2)
            percs_to_add = np.concatenate((incr_parts[0], percs_same, incr_parts[1]))
        case "inner":
            same_parts = np.array_split(percs_same, 2)
            percs_to_add = np.concatenate((same_parts[0], percs_incr, same_parts[1]))
        case "front":
            percs_to_add = np.concatenate([percs_incr, percs_same])
        case "back":
            percs_to_add = np.concatenate([percs_same, percs_incr])
        case _:
            raise ValueError(f"Unknown strategy: {strategy}")
    percs = (percs_base + percs_to_add).astype(int)
    assert np.sum(percs) == TEN_K
    return percs


T_GRID_SPEC = tuple[T_PERCS, T_PERCS, T_GRID]


@beartype
def make_grid(rows: T_POS_INT, cols: T_POS_INT, strategy: T_STRATEGY = "outer") -> T_GRID_SPEC:
    rows_perc = get_int_percs(rows, strategy=strategy)
    cols_perc = get_int_percs(cols, strategy=strategy)
    grid = np.arange(rows * cols).reshape(rows, cols)
    return (rows_perc, cols_perc, grid)


# recursive. no nulls or floats in this json spec
T_GRID_JSON = dict[str, "T_GRID_JSON"] | list["T_GRID_JSON"] | str | int | bool


@beartype
def grid_to_json_dict(grid_spec: T_GRID_SPEC, name: str) -> T_GRID_JSON:
    rows_perc, cols_perc, grid = grid_spec
    return {
        "uuid": str(uuid4()).upper(),
        "name": name,
        "type": "grid",
        "info": {
            "rows": rows_perc.size,
            "columns": cols_perc.size,
            "rows-percentage": rows_perc.tolist(),
            "columns-percentage": cols_perc.tolist(),
            "cell-child-map": grid.tolist(),
            "show-spacing": False,
            "spacing": 0,
            "sensitivity-radius": 20,
        },
    }

@beartype
def make_grid_json(rows: T_POS_INT, cols: T_POS_INT, strategy: T_STRATEGY = "outer", name: str = "") -> str:
    grid_spec = make_grid(rows, cols, strategy=strategy)
    return json.dumps(grid_to_json_dict(grid_spec, name), ensure_ascii=False, indent=2)



In [2]:
print(make_grid_json(2,3,name="test"))

{
  "uuid": "875CC104-2736-41B4-AD0F-15C2A5831E0D",
  "name": "test",
  "type": "grid",
  "info": {
    "rows": 2,
    "columns": 3,
    "rows-percentage": [
      5000,
      5000
    ],
    "columns-percentage": [
      3334,
      3333,
      3333
    ],
    "cell-child-map": [
      [
        0,
        1,
        2
      ],
      [
        3,
        4,
        5
      ]
    ],
    "show-spacing": false,
    "spacing": 0,
    "sensitivity-radius": 20
  }
}


In [4]:
import pyperclip

big_grid = make_grid_json(16,30,name="PetaGrid")
pyperclip.copy(big_grid)


In [13]:
grid_spec = make_grid_json()

BeartypeCallHintParamViolation: Function __main__.get_int_percs() parameter n_parts=-1 violates type hint typing.Annotated[int, Is[lambda x: x > 0]], as int -1 violates validator Is[lambda x: x > 0]:
    False == Is[lambda x: x > 0].

In [28]:
    with open("custom-layouts_reindexed.json", "w", encoding="utf-8") as output:
        json.dump(data, output, ensure_ascii=False, indent=2)

'1F465CA4-3CDC-4015-8B62-F00173BA2193'

In [3]:
grid = np.array(
    [
        [0, 1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35],
    ]
)

new_grid = reiterate_grid(grid)
new_grid

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

In [23]:
make_grid(17,5)[2].tolist()

[[0, 1, 2, 3, 4],
 [5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34],
 [35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44],
 [45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54],
 [55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64],
 [65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74],
 [75, 76, 77, 78, 79],
 [80, 81, 82, 83, 84]]

In [25]:
# unnecessary milp for fun
from scipy.optimize import LinearConstraint, Bounds, milp

n_parts = 17
perc = 10_000 / n_parts

bounds = Bounds(lb=np.floor(perc), ub=np.ceil(perc))
c = np.ones(n_parts)

b_u = np.array([10000])  # A = c, b_l = b_u
constraints = LinearConstraint(c, b_u, b_u)

integrality = c

res = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
res.x

array([588., 588., 588., 588., 588., 588., 588., 588., 588., 588., 588.,
       588., 588., 589., 589., 589., 589.])